In [1]:
import pandas as pd

# Define file paths for training data
train_beneficiary_path = r'C:\Users\melny\OneDrive\Desktop\Healthcare Provider Fraud Detction Analysis\Train_Beneficiarydata-1542865627584.csv'
train_inpatient_path = r'C:\Users\melny\OneDrive\Desktop\Healthcare Provider Fraud Detction Analysis\Train_Inpatientdata-1542865627584.csv'
train_outpatient_path = r'C:\Users\melny\OneDrive\Desktop\Healthcare Provider Fraud Detction Analysis\Train_Outpatientdata-1542865627584.csv'
train_path = r'C:\Users\melny\OneDrive\Desktop\Healthcare Provider Fraud Detction Analysis\Train-1542865627584.csv'

# Define file paths for test data
test_beneficiary_path = r'C:\Users\melny\OneDrive\Desktop\Healthcare Provider Fraud Detction Analysis\Test_Beneficiarydata-1542969243754.csv'
test_inpatient_path = r'C:\Users\melny\OneDrive\Desktop\Healthcare Provider Fraud Detction Analysis\Test_Inpatientdata-1542969243754.csv'
test_outpatient_path = r'C:\Users\melny\OneDrive\Desktop\Healthcare Provider Fraud Detction Analysis\Test_Outpatientdata-1542969243754.csv'
test_path = r'C:\Users\melny\OneDrive\Desktop\Healthcare Provider Fraud Detction Analysis\Test-1542969243754.csv'

# Load training datasets
train_beneficiary = pd.read_csv(train_beneficiary_path)
train_inpatient = pd.read_csv(train_inpatient_path)
train_outpatient = pd.read_csv(train_outpatient_path)
train = pd.read_csv(train_path)

# Load test datasets
test_beneficiary = pd.read_csv(test_beneficiary_path)
test_inpatient = pd.read_csv(test_inpatient_path)
test_outpatient = pd.read_csv(test_outpatient_path)
test = pd.read_csv(test_path)

def data_cleaning_beneficiary(df):
    # Fill missing values in the 'DOD' column with a placeholder date
    df['DOD'].fillna('2100-01-01', inplace=True)

    # Fill missing values for numerical columns with median
    numerical_cols_beneficiary = df.select_dtypes(include=['float64', 'int64']).columns
    df[numerical_cols_beneficiary] = df[numerical_cols_beneficiary].fillna(df[numerical_cols_beneficiary].median())

    # Fill missing values for categorical columns with mode
    categorical_cols_beneficiary = df.select_dtypes(include=['object']).columns
    df[categorical_cols_beneficiary] = df[categorical_cols_beneficiary].fillna(df[categorical_cols_beneficiary].mode().iloc[0])

    # Convert date columns to datetime type
    df['DOB'] = pd.to_datetime(df['DOB'])
    df['DOD'] = pd.to_datetime(df['DOD'])

    # Ensure categorical columns are of type 'category'
    categorical_cols = ['Gender', 'Race', 'RenalDiseaseIndicator', 'State', 'County']
    df[categorical_cols] = df[categorical_cols].astype('category')

    return df

def handle_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

def data_cleaning_general(df):
    numerical_columns = df.select_dtypes(include=['float64', 'int64']).columns
    for col in numerical_columns:
        df = handle_outliers_iqr(df, col)
    return df

def convert_date_columns(df, date_columns):
    for col in date_columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')
    return df

def convert_to_category(df, categorical_columns):
    df[categorical_columns] = df[categorical_columns].astype('category')
    return df

# Apply data cleaning to beneficiary datasets
train_beneficiary = data_cleaning_beneficiary(train_beneficiary)
test_beneficiary = data_cleaning_beneficiary(test_beneficiary)

# Apply data cleaning to inpatient and outpatient datasets
train_inpatient = data_cleaning_general(train_inpatient)
test_inpatient = data_cleaning_general(test_inpatient)

train_outpatient = data_cleaning_general(train_outpatient)
test_outpatient = data_cleaning_general(test_outpatient)

# Convert date columns for inpatient data
date_columns_inpatient = ['ClaimStartDt', 'ClaimEndDt', 'AdmissionDt', 'DischargeDt']
train_inpatient = convert_date_columns(train_inpatient, date_columns_inpatient)
test_inpatient = convert_date_columns(test_inpatient, date_columns_inpatient)

# Convert date columns for outpatient data
date_columns_outpatient = ['ClaimStartDt', 'ClaimEndDt']
train_outpatient = convert_date_columns(train_outpatient, date_columns_outpatient)
test_outpatient = convert_date_columns(test_outpatient, date_columns_outpatient)

# Convert categorical columns for inpatient data
categorical_columns_inpatient = ['AttendingPhysician', 'OperatingPhysician', 'OtherPhysician', 'ClmAdmitDiagnosisCode', 'DiagnosisGroupCode'] + [f'ClmDiagnosisCode_{i}' for i in range(1, 11)] + [f'ClmProcedureCode_{i}' for i in range(1, 7)]
train_inpatient = convert_to_category(train_inpatient, categorical_columns_inpatient)
test_inpatient = convert_to_category(test_inpatient, categorical_columns_inpatient)

# Convert categorical columns for outpatient data
categorical_columns_outpatient = ['AttendingPhysician', 'OperatingPhysician', 'OtherPhysician', 'ClmAdmitDiagnosisCode'] + [f'ClmDiagnosisCode_{i}' for i in range(1, 11)] + [f'ClmProcedureCode_{i}' for i in range(1, 7)]
train_outpatient = convert_to_category(train_outpatient, categorical_columns_outpatient)
test_outpatient = convert_to_category(test_outpatient, categorical_columns_outpatient)

# Verify Train_Beneficiarydata
print("Missing values in Train_Beneficiarydata after all steps:")
print(train_beneficiary.isnull().sum())
print("Data types in Train_Beneficiarydata after all steps:")
print(train_beneficiary.dtypes)

# Verify Test_Beneficiarydata
print("Missing values in Test_Beneficiarydata after all steps:")
print(test_beneficiary.isnull().sum())
print("Data types in Test_Beneficiarydata after all steps:")
print(test_beneficiary.dtypes)

# Verify Train_Inpatientdata
print("Missing values in Train_Inpatientdata after all steps:")
print(train_inpatient.isnull().sum())
print("Data types in Train_Inpatientdata after all steps:")
print(train_inpatient.dtypes)

# Verify Test_Inpatientdata
print("Missing values in Test_Inpatientdata after all steps:")
print(test_inpatient.isnull().sum())
print("Data types in Test_Inpatientdata after all steps:")
print(test_inpatient.dtypes)

# Verify Train_Outpatientdata
print("Missing values in Train_Outpatientdata after all steps:")
print(train_outpatient.isnull().sum())
print("Data types in Train_Outpatientdata after all steps:")
print(train_outpatient.dtypes)

# Verify Test_Outpatientdata
print("Missing values in Test_Outpatientdata after all steps:")
print(test_outpatient.isnull().sum())
print("Data types in Test_Outpatientdata after all steps:")
print(test_outpatient.dtypes)

# Verify Train
print("Missing values in Train after all steps:")
print(train.isnull().sum())
print("Data types in Train after all steps:")
print(train.dtypes)

# Verify Test
print("Missing values in Test after all steps:")
print(test.isnull().sum())
print("Data types in Test after all steps:")
print(test.dtypes)



Missing values in Train_Beneficiarydata after all steps:
BeneID                             0
DOB                                0
DOD                                0
Gender                             0
Race                               0
RenalDiseaseIndicator              0
State                              0
County                             0
NoOfMonths_PartACov                0
NoOfMonths_PartBCov                0
ChronicCond_Alzheimer              0
ChronicCond_Heartfailure           0
ChronicCond_KidneyDisease          0
ChronicCond_Cancer                 0
ChronicCond_ObstrPulmonary         0
ChronicCond_Depression             0
ChronicCond_Diabetes               0
ChronicCond_IschemicHeart          0
ChronicCond_Osteoporasis           0
ChronicCond_rheumatoidarthritis    0
ChronicCond_stroke                 0
IPAnnualReimbursementAmt           0
IPAnnualDeductibleAmt              0
OPAnnualReimbursementAmt           0
OPAnnualDeductibleAmt              0
dtype: int64
Data 